This Jupyter Notebook is used for extracting data from multiple sources. The sources include trip data, occupancy data, and parking lot data. 

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely import wkt
import numpy as np
import os
import folium
from pathlib import Path
import boto3

# import from src
os.chdir('../src')
from trips_data_retrieval import get_agg_trips_by_market, get_agg_trips
import inrix_data_science_utils.maps.quadkey as qkey
from utils import *

# set paths
data_path = Path('../data')

### Extract the 6 Flags Polygon Shapes
#### This might be unnecessary

In [ ]:
from polygon_retrieval import get_polygons

# Six Flags
brand_id = 'SG_BRAND_0dd52fbf1cd77fc38e06650435ada07d'
poly_file_name = data_path / 'six_flags_polygons.csv'

In [ ]:

if not os.path.exists(poly_file_name):
    polygon_df = get_polygons(brand_id, format='df', echo_query=True)
    relevant_cols = ['location_name', 'top_category', 'sub_category', 'latitude', 'longitude', 'street_address', 'city', 'region', 'postal_code', 'open_hours',
                 'category_tags', 'polygon_wkt', 'polygon_class', 'enclosed', 'is_synthetic', 'includes_parking_lot', 'wkt_area_sq_meters', 'tz', 'year',
                 'month', 'pg_id', 'parking_wkt']
    polygon_df = polygon_df[relevant_cols]
    polygon_df.to_csv(poly_file_name, index=False)
else:
    polygon_df = pd.read_csv(poly_file_name)

In [ ]:
# make a new df with just the unique latitude and longitude columns
store_locations_df = polygon_df[['latitude', 'longitude', 'street_address', 'city', 'region']].drop_duplicates()
display(store_locations_df)

six_flag_stores = folium.Map(zoom_start=15)
for i, row in store_locations_df.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        color='blue',
        radius=4
    ).add_to(six_flag_stores)
six_flag_stores.fit_bounds(six_flag_stores.get_bounds())
six_flag_stores

In [ ]:
# city = 'Oklahoma City'
# test_site = store_locations_df[store_locations_df['city'] == city].iloc[0]
# test_point = test_site['latitude'], test_site['longitude']

# # get the quadkey for a specific location
# qk = qkey.QuadKey('0')
# qk = qk.from_geo(test_point, level=16)
# print(qk)
# qk.show()

#### Map the parking lots at each store

In [ ]:
lot_gpd = polygon_df.copy()
geometry = lot_gpd['parking_wkt'].apply(wkt.loads)
lot_gpd = gpd.GeoDataFrame(lot_gpd, crs='epsg:4326', geometry=geometry)
print(lot_gpd.crs)
lot_gpd[lot_gpd['city'] == city].plot(figsize=(6, 6))
plt.show()

In [ ]:
lot_map = folium.Map(zoom_start=10)
cities = lot_gpd['city'].unique()
# add pois to lot map
for city in cities:
    city_gpd = lot_gpd[lot_gpd['city'] == city].copy()
    geom = city_gpd['polygon_wkt'].values[0]
    # convert geom to a shapely geometry object
    geom = wkt.loads(geom)
    sim_geo = gpd.GeoSeries(geom)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {'fillColor': 'red', 'color': 'red', 'opacity': 0.3})
    folium.Popup(city).add_to(geo_j)
    geo_j.add_to(lot_map)

# add parking lots to lot map
for _, r in lot_gpd.iterrows():
    geom = r['geometry']  # this needs to be a shapely geometry object
    sim_geo = gpd.GeoSeries(geom)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'opacity': 0.8})
    geo_j.add_to(lot_map)
print('loop done')
# lot_map

# neighbors = qk.nearby()
# neighbors.append(str(qk))
# lot_map = add_qks_to_map(lot_map, neighbors)
qk.show(lot_map)
lot_map

### Get the Parking Occupancy Ground Truth Data

In [2]:
from parking_data_retrieval import get_parking_data

Find which cities have the most parking data

In [3]:
destinations = pd.read_csv('../data/valid_parking_destinations.csv')
# sort destinations by num_lots
destinations = destinations.sort_values('num_lots', ascending=False)
display(destinations.head(25))

,pk_destination,str_name,num_lots
19,8faeea56-7a52-11df-ad8f-6b84bf453d3d,Santa Monica,15
16,8faca340-7a52-11df-8a97-875256aaa997,Irvine,11
10,8f98cf00-7a52-11df-bb23-6f961d7daa51,Ann Arbor,9
6,8f8ae430-7a52-11df-93fb-6b6334ed9a2a,Knoxville,7
15,8fa9dfde-7a52-11df-af59-273aa13aeea1,Phoenix,6
14,8fa8608c-7a52-11df-841d-870c5e91ead8,Madison,5
3,8f854570-7a52-11df-9a45-93d95744a90c,Asheville,5
18,8fae86e2-7a52-11df-b2bb-d72f2aaf97c2,Sacramento,4
5,8f8935cc-7a52-11df-85ef-037cba85cd49,Philadelphia,4
9,8f91a77a-7a52-11df-844f-c76b5ffdf7f2,Iowa City,4


In [6]:
destination_name = 'Irvine'
datetime_start = '2023-01-01'
datetime_end = '2023-01-31'

parking_filename = f'retry_parking_{destination_name}_{datetime_start}_{datetime_end}.csv'

In [7]:
if not os.path.exists(data_path / parking_filename):
    parking_df = get_parking_data(pk_lot=None,
                                  destination_name=destination_name,
                                  datetime_start=datetime_start,
                                  datetime_end=datetime_end,
                                  echo_query=True)
    relevant_cols = ['dt_start_date', 'dt_end_date', 'i_avail', 'i_occ', 'f_pct_occ', 'pk_lot_alias',
                     'f_occupancy_rank', 'pk_lot', 'location', 'geometry']
    parking_df = parking_df[relevant_cols]
    parking_df.to_csv(data_path / parking_filename, index=False)
else:
    parking_df = pd.read_csv(data_path / parking_filename)
    parking_df['dt_start_date'] = pd.to_datetime(parking_df['dt_start_date'])
print(parking_df.shape)
parking_df.head()


        SELECT  lot_occupancy.*, st_astext(lot.gpnt_location) as location, st_astext(lot.mpoly_lot) as geometry
        FROM lot_occupancy
        JOIN lot on lot_occupancy.pk_lot = lot.pk_lot
        JOIN destination on lot.pk_city = destination.pk_destination
        WHERE dt_start_date AT TIME ZONE str_timezone >=  '2023-01-01'
            AND dt_start_date AT TIME ZONE str_timezone <  '2023-01-31'
            AND f_pct_occ IS NOT NULL
            AND destination.pk_country = 'b363bb38-ca10-11e1-9278-12313d1b6657' -- USA
            AND destination.str_name = 'Irvine'
                   
        


OperationalError: could not connect to server: Connection timed out (0x0000274C/10060)
	Is the server running on host "devdb-zzz.parkme.com" (10.104.17.126) and accepting
	TCP/IP connections on port 5432?


In [ ]:
parking_gpd = parking_df.copy()
geometry = parking_gpd['geometry'].apply(wkt.loads)
parking_gpd = gpd.GeoDataFrame(parking_gpd, crs='epsg:4326', geometry=geometry)

In [ ]:
# get the quadkey for a specific location
lot_gpd = parking_gpd.groupby('pk_lot').agg({'geometry': 'first'}).reset_index()
qk = qkey.QuadKey('0')
level = 15
lot_gpd['centroid'] = lot_gpd['geometry'].apply(lambda x: x.centroid)
lot_gpd['center_lat'] = lot_gpd['centroid'].apply(lambda x: x.coords[0][1])
lot_gpd['center_lon'] = lot_gpd['centroid'].apply(lambda x: x.coords[0][0])
lot_gpd['qk'] = lot_gpd.apply(lambda row: qk.from_geo((row['center_lat'], row['center_lon']), level=level), axis=1)
display(lot_gpd)

In [ ]:
# folium plot the parking lots
m = folium.Map(zoom_start=10)

for _, r in lot_gpd.iterrows():
    geom = r['geometry']  # this needs to be a shapely geometry object
    sim_geo = gpd.GeoSeries(geom)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'opacity': 0.8},
                           tooltip=r['pk_lot'])

    geo_j.add_to(m)


for q in lot_gpd['qk'].unique():
    polygon = folium.GeoJson(data=q.get_bounds_as_geojson(),
                             style_function=lambda x: {'fillColor': 'red', 'color': 'red', 'opacity': 0.3},
                             tooltip=str(q))
    polygon.add_to(m)


m

#### Parking Occupancy Initial Plots

In [ ]:
print(parking_df['pk_lot'].unique())

In [ ]:
# line graph of f_pct_occ vs. dt_start_date grouped by pk_lot
day_df = parking_df[(parking_df['dt_start_date'] >= '2023-01-01') &
                    (parking_df['dt_start_date'] <= '2023-01-7') &
                    (parking_df['pk_lot'] == 209634)]
# order day_df by dt_start_date
day_df = day_df.sort_values('dt_start_date')
display(day_df.head(5))

plt.figure(figsize=(10, 6))
day_df.set_index('dt_start_date', inplace=True)
day_df.groupby('pk_lot')['f_pct_occ'].plot(legend=True)
# plt.plot(day_df['dt_start_date'], day_df['f_pct_occ'])
plt.yticks(np.arange(0, 110, 10))
plt.ylabel('Percent Lot Occupancy')
plt.legend(title='Parking Lot')
plt.ylim(-1, 105)
plt.show()


Smooth into 1 minute buckets

In [ ]:
parking_df['dt_start_date'] = pd.to_datetime(parking_df['dt_start_date'])
print(parking_df['dt_start_date'].dtype)

In [ ]:
smooth_df = parking_df.sort_values('dt_start_date').copy()
smooth_df['time_bucket'] = smooth_df['dt_start_date'].dt.floor('min')
agg_dict = {'f_pct_occ': 'mean', 'i_avail': 'mean', 'i_occ': 'mean',
            'location': 'first', 'geometry': 'first',}
grouped_df = smooth_df.groupby(['pk_lot', 'time_bucket']).agg(agg_dict).reset_index()
# how many rows from the original go into each group of grouped_df

a, b = smooth_df.shape[0], grouped_df.shape[0]
print(f'At least {a - b} rows out of {a} from the original\ndf go into each group of grouped_df for a final size of {b}')
display(grouped_df)

In [ ]:
grouped_df.to_csv(data_path / 'IrvineSpectrumCenter_parking.csv', index=False)

In [ ]:
sanity_check = grouped_df[grouped_df['pk_lot'] == 209634] # grouped_df
sanity_check = sanity_check.sort_values('time_bucket')
# display(sanity_check)

plt.figure(figsize=(10, 6))
sanity_check.set_index('time_bucket', inplace=True)
sanity_check.groupby('pk_lot')['f_pct_occ'].plot(legend=True)
# plt.plot(day_df['dt_start_date'], day_df['f_pct_occ'])
plt.yticks(np.arange(0, 110, 10))
plt.ylabel('Percent Lot Occupancy')
plt.legend(title='Parking Lot')
plt.ylim(-1, 105)
plt.show()

### Extracting Trips

In [ ]:
# Change these as needed
# qk_list = ['02123003']  # seattle area
# qk_list = ['021230032110033003', '021230032110033012', '021230032110033021', '021230032110033030']  # QFC North Bend
# qk_list = [str(qk)]  # Six Flags OKC
# qk_list = neighbors  # if we want to get all the neighbors data too
qk_list = ['023013202100232', '023013202100233', '023013202102010', '023013202102011'] # 
start_date = pd.to_datetime('2023-01-01 00:00:00')
end_date = pd.to_datetime('2023-1-31 00:01:00')
trip_type = 'orig' # 'orig' or 'dest'
filename_description = 'IrvineSpectrumCenter'

# don't change these
start_year = str(start_date.year)
start_month = str(start_date.month).zfill(2)
start_day = str(start_date.day).zfill(2)
end_year = str(end_date.year)
end_month = str(end_date.month).zfill(2)
end_day = str(end_date.day).zfill(2)
out_filename = f'''{trip_type}_trips_{filename_description}_{start_year}-{start_month}-{start_day}_to_{
                end_year}-{end_month}-{end_day}.csv'''
out_file_path = data_path / out_filename
print('Will save the csv to ', out_file_path)
origin_qk = (trip_type == 'orig')

table_name = 'tripdata_na_restricted'
s3_tapp_data_dir = 'data-restricted'
s3_tapp_region = 'NA'

In [ ]:
# if out_file_path not in the data folder, save it there
if not os.path.exists(out_file_path):
    trips_df = get_agg_trips(
        agg_file_path = out_file_path,
        echo_query=True,
        table_name=table_name,
        start_date=start_date,
        end_date=end_date,
        qk_filter_list=qk_list,
        s3_tapp_data_dir=s3_tapp_data_dir,
        s3_tapp_region=s3_tapp_region,
        origin_qk=origin_qk,
    )
    print('Saved the csv to', out_file_path)
else:
    trips_df = pd.read_csv(out_file_path)

trips_df['date'] = pd.to_datetime(trips_df[['year', 'month', 'day', 'hour', 'minute', 'second']])
display(trips_df)

In [ ]:
# m = lot_map
# m = folium.Map(zoom_start=10)
map = m

N = 10
for i in range(0, trips_df.shape[0], N):
    folium.CircleMarker(
        location=[
            trips_df['end_lat'].values[i],
            trips_df['end_lon'].values[i]
        ],
        color='red',
        radius=1
    ).add_to(map)
# m.fit_bounds(m.get_bounds())

# for i in range(0, trips_df.shape[0], N):
#     folium.CircleMarker(
#         location=[
#             trips_df['start_lat'].values[i],
#             trips_df['start_lon'].values[i]
#         ],
#         color='blue',
#         radius=1
#     ).add_to(map)
# m.fit_bounds(m.get_bounds())

map